In [1]:
from fal import FalDbt
import os

In [3]:
cwd = os.getcwd()
faldbt = FalDbt(profiles_dir=cwd, project_dir=cwd)

faldbt.list_sources()
# [['results', 'ticket_data_sentiment_analysis']]

# faldbt.list_models()
# # {
# #   'zendesk_ticket_metrics': <RunStatus.Success: 'success'>, 
# #   'stg_o3values': <RunStatus.Success: 'success'>, 
# #   'stg_zendesk_ticket_data': <RunStatus.Success: 'success'>, 
# #   'stg_counties': <RunStatus.Success: 'success'>
# # }

[['airbnb', 'calendar'],
 ['airbnb', 'neighbourhoods'],
 ['airbnb', 'reviews'],
 ['airbnb', 'listings'],
 ['tpch', 'orders'],
 ['tpch', 'customer'],
 ['tpch', 'lineitem'],
 ['tpch', 'nation'],
 ['tpch', 'part'],
 ['tpch', 'partsupp'],
 ['tpch', 'region'],
 ['tpch', 'supplier']]